In [1]:
import numpy as np
import cvxopt
import matplotlib.pyplot as plt

# Soft margin SVM
this part we are going to implement soft margin svm. We're using cvxopt library to solve quadratic equations in support vector machines.

## formulas and equations
First we will speak of the CVXOPT library. in this library to solve a quadratic equation, equations must be in the form as equation \ref{eq:quadratic-cvxopt-form}
\begin{equation} 
\begin{aligned}
&min \: \frac{1}{2} x^T P x + q^Tx \\
&subject \; to \; Gx \leq h \\
& Ax=b
\end{aligned}
\label{eq:quadratic-cvxopt-form}
\end{equation}

And the dual problem equations in support vector machines are as
\begin{equation} \label{eq:dual-problem-expression}
max \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i,j=1}^{m} y_i y_j \alpha_i \alpha_j x_{i}^{T} x_j
\end{equation}
And if we write $H$ as $H_{i,j} = y_i y_j x_{i}^{T} x_j$
the optimazation equation will be as 
\begin{equation}
\begin{aligned}
&max \: \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \alpha^T H \alpha \\
& subject \; to \; 0 \leq \alpha_i \leq C \\
& \sum_{i}^{m} \alpha_i y_i = 0
\end{aligned}
\end{equation}
Note that $H$ will be our gram matrix. and if we want to convert our problem to the notation of CVXOPT library we can write

 $P$ stands for H with the shape $m \times m$, $q$ stands for -1 vector with -1 values and with shape of $m \times 1$, $G$ stands for a digonal matrix with -1 on the diagonal and the shape of $m \times m$, $h$ stands for a zero vector with the shape of $m \times 1$, $A$ stands for the label vector y with the shape of $m \times 1$ and $b$ is a scalar.

To convert the maximum problem into minimum we can multilpy all equations with a $-1$ 
\begin{equation}
\begin{aligned}
& min \: \frac{1}{2} \alpha^T H \alpha - 1^T \alpha_i \\
& subject \; to \; \alpha_i \leq C \\
&  -\alpha_i \leq 0 \\
& \sum_{i}^{m} \alpha_i y_i = 0
\end{aligned}
\end{equation}
also we can rewrite the last equation as below, meaning all lables ($y$) are in one vector
\begin{equation}
y^T \alpha = 0
\end{equation}

## implementation
Now we're going to implement soft margin svm with an example.  The data are as follows
\begin{equation}
X =
\begin{pmatrix}
3 & 1 & 2 & 6 & 7 & 5 & 2 \\
4 & 4 & 3 & -1 & -1 & -3 & 4
\end{pmatrix}
\end{equation}
And the labels are 
\begin{equation}
Y = 
\begin{pmatrix}
-1 & -1 & -1 & 1 & 1 & 1 & 1 \\
\end{pmatrix}
\end{equation}
because the dimension of samples are 2 and we have two constraints $G$ and $h$, we can write these constraints as 
\begin{equation}
G = 
\begin{pmatrix}
-1 & 0 \\
0 & -1 \\
1 & 0 \\
0 & 1
\end{pmatrix}
\end{equation}

\begin{equation}
h = 
\begin{pmatrix}
0 \\
0 \\
C \\
C
\end{pmatrix}
\end{equation}

In [2]:
def svm_soft_margin(X, y, C):
    """
    implementation of soft margin svm
    
    INPUTS:
    --------
    X: the numpy matrix of input vectors, shape is m*n meaning m is the dimension and n is data sample size
    y: numpy vector of labels, shape is n*1
    
    """
    
    m,n = X.shape
    
    ## multiplying the labels with inputs
    X_dash = y * X

    ## inner product 
    ## with this the labels will be multiplied twice as in the dual problem equation
    H = np.dot(X_dash, X_dash.T)
    
    ## convert data into cvxopt format
    P = cvxopt.matrix(H)
    q = cvxopt.matrix(-np.ones(m))
    G = cvxopt.matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
    h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt.matrix(y.reshape(1, -1))
    b = cvxopt.matrix(np.zeros(1))

    ## solve the qaudratic equations
    answer = cvxopt.solvers.qp(P, q, G, h, A, b)

    ## the lagrangian coefficients (named as alpha)
    alphas = np.array(answer['x'])

    ## computing parameters of the line equation
    w = ((y * alphas).T @ X).reshape(-1,1)
    S = (alphas > 1e-4).flatten()
    b = y[S] - np.dot(X[S], w)

    #Display results
    print('Alphas = ',alphas[alphas > 1e-4])
    print('w = ', w.flatten())
    print('b = ', b[0])


In [3]:
## dataset
X = np.array([[3,4],[1,4],[2,3],[6,-1],[7,-1],[5,-3],[2,4]], dtype=float)
y = np.array([-1,-1, -1, 1, 1 , 1, 1 ], dtype=float)

## define C 
C = 10

y = y.reshape(-1,1)
svm_soft_margin(X, y, C)

     pcost       dcost       gap    pres   dres
 0: -1.5556e+01 -2.5999e+02  3e+02  1e-01  2e-14
 1: -1.8050e+01 -3.9497e+01  2e+01  4e-03  1e-14
 2: -2.1437e+01 -2.3412e+01  2e+00  3e-04  2e-14
 3: -2.2496e+01 -2.2997e+01  5e-01  3e-05  1e-14
 4: -2.2561e+01 -2.2568e+01  6e-03  3e-07  2e-14
 5: -2.2562e+01 -2.2563e+01  6e-05  3e-09  2e-14
 6: -2.2562e+01 -2.2563e+01  6e-07  3e-11  1e-14
Optimal solution found.
Alphas =  [4.9999998  6.31250013 1.31249982 9.99999997]
w =  [ 0.25000001 -0.25000001]
b =  [-0.74999997]
